# Dependency Grammar (first assignment NLU)


* Student name: Gaia Trebucchi
* Student number: 224464




First, we load `en_core_web_sm` with `spacy.load`. This will return a `Language` object stored as `spacy_nlp` containing all components and data needed to process text.

In [1]:
import spacy

spacy_nlp = spacy.load('en_core_web_sm')

sentence='I saw the man with the telescope.'
sentence1='Gaia brought her cat Costina some delicious food'

### Function 1:
#### Extract a path of dependency relations from the ROOT to a token. 
* **input**: A sentence passed as a string 
* **output**: A dictionary whose keys are all the tokens of the input sentence and the value for each key is a list of tuples, where the first element of the tuple is a token and the second element is the dependency relation between the token and its head in the sentence parsing. The list, for each key, represents the path from the root of the sentence to the token stored as the key.   

First, the sentence is processed through `spacy_nlp(sentence)` to obtain a `Doc` object. Then a dictionary is created to store the dependency relation paths from the root to each token. 
The function cycles through all the `Token` objects of the `Doc` in this way: 
* each token is added as first element of its relative `dep_path` in a tuple whose second element is the dependency relation with its head
* a while loop adds to the relative path all the tuples (token, dependency relation) that we encounter by ascending  the dependencies from the token we are considering. This step is done by calling the head of the token until we found the root of the sentence. To state that, the stopping criterium for the while loop is a check of the token dependency, if it results equal to `'ROOT'` the function exits the while. Before adding the relative path for each token to the dictionary, the path is reversed to follow the descending order from the root to the token. The choice of first computing the path in the ascending way and then reverse it it's due to the fact that each token owns a unique head, on the contrary it could have more than one child and the search of the descending path in that case would need more than one iteration across the sentence. 




In [2]:
def path_dependency(sentence):
    doc=spacy_nlp(sentence)
    list_path=dict()
    for token in doc:
        tok=token
        dep_path=[(tok,tok.dep_)]
        while tok.dep_!='ROOT':
            tok=tok.head
            dep_path.append((tok,tok.dep_))
        path=dep_path[::-1]
        list_path[token]=path
    return list_path

Example with the sentence "I saw the man with the telescope":

In [3]:
print(path_dependency(sentence))

{I: [(saw, 'ROOT'), (I, 'nsubj')], saw: [(saw, 'ROOT')], the: [(saw, 'ROOT'), (man, 'dobj'), (the, 'det')], man: [(saw, 'ROOT'), (man, 'dobj')], with: [(saw, 'ROOT'), (man, 'dobj'), (with, 'prep')], the: [(saw, 'ROOT'), (man, 'dobj'), (with, 'prep'), (telescope, 'pobj'), (the, 'det')], telescope: [(saw, 'ROOT'), (man, 'dobj'), (with, 'prep'), (telescope, 'pobj')], .: [(saw, 'ROOT'), (., 'punct')]}


### Fuction 2:
#### Extract subtree of a dependents given a token.
* **input**: a sentence passed as string
* **output**: a dictionary whose keys are all the tokens of the input sentence and whose value for each key is the list of tokens belonging to the subtree of the key token, in the order they appear in the sentence.

First, the sentence is processed through `spacy_nlp(sentence)` to obtain a `Doc` object. Then a dictionary is created to store the subtree of each token.
The function cycles through all the `Token` objects in the `Doc` in this way:
* the subtree of the token is extracted through the attribute `Token.subtree` that returns the subtree containing the token and all the token's syntactic descendants.
* A list is created and each token of the subtree is added
* The token is added to the dictionary as key and its subtree list is added as value

In [4]:
def subtree_token(sentence):
    doc=spacy_nlp(sentence)
    sub_token=dict()
    for token in doc:
        depend=[]
        sub=token.subtree
        for t in sub:
            depend.append(t)
        sub_token[token]=depend
    return sub_token

Example with the sentence "I saw the man with the telescope":

In [5]:
print(subtree_token(sentence))

{I: [I], saw: [I, saw, the, man, with, the, telescope, .], the: [the], man: [the, man, with, the, telescope], with: [with, the, telescope], the: [the], telescope: [the, telescope], .: [.]}


### Function 3:
#### check if a given list of tokens (segment of a sentence) forms a subtree.
* **input**: a sentence passed as a string and a segment passed as a list of strings
* **output**: a `Boolean` that states if the segment forms a valid subtree of dependency in the sentence

First, the function `subtree_token` defined above (Function 2) is used to extract all the dependency subtrees of the sentence, stored from the previous function as the values of a dictionary.
Then, the function cycles across all the values of the dictionary (all the subtrees) in that way:
* The first check is on the length of the sentence and the number of elements of the subtree: if they don't match, the function skip to the next subtree without comparing the segment with the subtree components
* If the length match, the function creates a list with the text of the subtree tokens and compares it with the segment. If they are equivalent the segment forms a valid subtree and True is returned, otherwise the function proceeds to the successive value. 

At the end of the for cycle, if there isn't any subtree that matches the input segment, False is returned.

In [6]:
 def check_subtree(sentence,segment):
    sub_tree=subtree_token(sentence)
    for value in sub_tree.values():
        if len(value)==len(segment):
            text_list=[]
            for token in value:
                text_list.append(token.text)
            if text_list==segment:
                return True
    return False


Example with two different segments (one that forms a subtree of dependencies in the input sentence parsing and one that doesn't) and the sentence "I saw the man with the telescope":

In [7]:
print(check_subtree(sentence,[ 'the', 'man', 'with']))
print(check_subtree(sentence,[ 'the', 'man','with','the', 'telescope']))

False
True


### Function 4:
#### identify head of a span, given its tokens.
* **input**: a list of tokens (not necessarily a sentence)
* **output**: the token representing the head of the span formed by the input tokens

First, the function converts the list of tokens in a single string, that is then processed by `spacy_nlp(segment_string)` to produce a `Doc` object. Then a `Span` that covers the entire segment is created and the head of the span is returned by `span.root` attribute.

In [8]:
def head_of_span(segment):
    segment_string=segment[0]
    for i in range(1,len(segment)):
        segment_string+=" "+segment[i]
    doc=spacy_nlp(segment_string)
    span=doc[:]
    return span.root
            

Example with different lists of tokens:

In [9]:
print(head_of_span(['the', 'man','with','the','telescope']))
print(head_of_span(['last','chance','for','you']))


man
chance


### Function 5:
#### extract sentence subject, direct object and indirect object spans.
* **input**: a sentence passed as string
* **output**: a dictionary whose keys are tuples consisting of the token and its dependency relation and the value for each key is the span of the token.

First, the sentence is processed with `spacy_nlp(sentence)` to obtain a `Doc` object and a dictionary is created to store the dependencies we are focusing on and the relative spans. 
The function cycles through all the `Token` object of the `Doc` in this way:
* The first step consists of stating if the dependency of the token under observation matches one of the dependency we are interested in (the dependency should be `'nsubj'` for the subject, `'dobj'` for the direct object and `'dative'` for the indirect object).
* When a subject, a direct object or an indirect object token is found the function creates a `Span` object of the subtree having the token as root. This is done by using the attribute `token.left_edge.i` and `token.right_edge.i` that provides respectively the index of the first and the index of the last token of the subtree, that are used to create a `Span` object for the syntactic phrase we are interested in.
* The last step consists of adding to the dictionary a tuple (token, dependency of the token) as key and the list of the span found at the previous step as value. The choice of storing as key this tuple has been made to highlight the head of the span (i.e. the token having as dependency 'nsubj', 'dobj' or 'dative').


In [10]:
def get_spans(sentence):
    doc=spacy_nlp(sentence)
    spans_dict=dict()
    for token in doc:
        if token.dep_=='nsubj' or token.dep_=="dobj" or token.dep_=="dative":
            span=doc[token.left_edge.i:token.right_edge.i+1]
            spans_dict[(token,token.dep_)]=[span]
    return spans_dict

Example with the two sentences: "I saw the man with the telescope", "Gaia brought her cat Costina some delicious food":

In [12]:
print(get_spans(sentence))
print(get_spans(sentence1))

{(I, 'nsubj'): [I], (man, 'dobj'): [the man with the telescope]}
{(Gaia, 'nsubj'): [Gaia], (Costina, 'dative'): [her cat Costina], (food, 'dobj'): [some delicious food]}
